In [18]:
pip install jsonlines

In [27]:
import pandas as pd
import jsonlines
import re
from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer

In [20]:
def read_jsonl_to_dataframe(file_path):
    data = []
    with jsonlines.open(file_path) as reader:
        for line in reader:
            data.append(line)
    df = pd.DataFrame(data)
    return df

In [33]:
train_df = read_jsonl_to_dataframe('/content/arabic_train.jsonl')
val_df = read_jsonl_to_dataframe('/content/arabic_val.jsonl')
test_df = read_jsonl_to_dataframe('/content/arabic_test.jsonl')

In [34]:
train_df.head()

,id,url,title,summary,text
0,140323_russian_troops_crimea_naval_base,https://www.bbc.com/arabic/worldnews/2014/03/1...,القوات الأوكرانية تبدأ الانسحاب من القرم,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينو...
1,130528_egypt_nile_dam,https://www.bbc.com/arabic/middleeast/2013/05/...,هل يفرض سد النهضة الإثيوبي واقعا جديدا على مصر؟,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...",بحلول عام 2050 ستحتاج مصر إلى 21 مليار متر مكع...
2,world-47242349,https://www.bbc.com/arabic/world-47242349,تعرف على منطقة كشمير التي تسببت بحربين بين اله...,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,وذكرت وكالة الأنباء المحلية (جي.إن.إس) أن جماع...
3,vert-cul-55078328,https://www.bbc.com/arabic/vert-cul-55078328,ماذا تعرف عن العالم الخفي للمعابد اليابانية ال...,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ووقع اختياره على واد عمقه 800 متر محاط بثماني ...
4,141023_yemen_hodeida,https://www.bbc.com/arabic/middleeast/2014/10/...,"اشتباك بين الحوثيين و""الحراك التهامي"" في الحدي...","أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",مسلح حوثي في إب وقال المصدر إن المسلحين الحوثي...


In [35]:
test_df.head()

,id,url,title,summary,text
0,130806_nidhal_hassan_trial,https://www.bbc.com/arabic/worldnews/2013/08/1...,نضال حسن يمثل أمام محكمة عسكرية أمريكية,تنظر محكمة عسكرية أمريكية في وقت لاحق من اليوم...,نضال حسن واعترف نضال حسن، الذي يدافع عن نفسه، ...
1,160129_germany_asylum_seekers,https://www.bbc.com/arabic/worldnews/2016/01/1...,ألمانيا تسعى للحد من الهجرة بإعلان الجزائر وتو...,كشفت ألمانيا النقاب عن خطط لإضافة الجزائر والم...,ألمانيا تواجه مصاعب في التعامل مع الأعداد المت...
2,130729_syria_homs_area_retaken,https://www.bbc.com/arabic/middleeast/2013/07/...,الخالدية : التليفزيون السوري يعلن استعادة الجي...,قال التليفزيون السوري إن قوات الحكومة استعادت ...,"وأكدت وسائل الإعلام السورية أن الجيش ""استعاد ا..."
3,140517_arsenal_fa_cup_winners,https://www.bbc.com/arabic/sports/2014/05/1405...,الارسنال يتوج بلقب كأس انكلترا,توج فريق الارسنال ببطولة كأس انجلترا لكرة القد...,وفاجأ هال سيتي الحضور بمباغتة الارسنال بهدفين ...
4,140722_iraq_minorities,https://www.bbc.com/arabic/middleeast/2014/07/...,العراق: الأقليات في سهل نينوى,يوضع الصراع في العراق غالبا في إطار صراع بين ا...,معبد يزيدي في سهول محافظة نينوى يعيش المسيحيون...


In [36]:
val_df.head()

,id,url,title,summary,text
0,140921_yemen_pm_resign,https://www.bbc.com/arabic/middleeast/2014/09/...,اتفاق لتشكيل حكومة جديدة بين الحوثيين وعدة احز...,وقع الحوثيون اتفاقا مع عدد من الاحزاب اليمنية ...,انضم بعض عناصر القوات المسلحة الى الحوثيين في ...
1,inthepress-56197077,https://www.bbc.com/arabic/inthepress-56197077,مظاهرات تونس: إلى أي مدى سيصل الصراع بين أطراف...,ناقش معلقون في صحف عربية تطور الأزمة السياسية ...,الرئيس التونسي قيس سعيد ورئيس الحكومة هشام الم...
2,151114_gatwick_airport_frenchman_charged,https://www.bbc.com/arabic/worldnews/2015/11/1...,اتهام رجل بحيازة أسلحة بعد إعلان حالة تأهب في ...,اتهمت الشرطة البريطانية رسميا رجلا فرنسيا بحيا...,المسافرون انتظروا ست ساعات على أرض مطار غاتويك...
3,media-48880286,https://www.bbc.com/arabic/media-48880286,عالم سري للسوريّات على فيسبوك محظور على الرجال...,تطلب شابة على إحدى صفحات فيسبوك المغلقة نصيحة ...,وانهالت تعليقات النساء على هذه المداخلة بين مؤ...
4,130311_us_nkorea,https://www.bbc.com/arabic/middleeast/2013/03/...,الولايات المتحدة توسع العقوبات ضد كوريا الشمالية,فرضت الولايات المتحدة عقوبات على بنك العملة ال...,قطعت كوريا الشمالية الخط الهاتفي الساخن بين ال...


In [37]:
train_df=train_df.drop(['id', 'url', 'title'], axis=1)
test_df=test_df.drop(['id', 'url', 'title'], axis=1)
val_df=val_df.drop(['id', 'url', 'title'], axis=1)

In [39]:
train_df.head()

,summary,text
0,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,وكان الرئيس الأوكراني المؤقت، الكسندر تورتشينو...
1,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...",بحلول عام 2050 ستحتاج مصر إلى 21 مليار متر مكع...
2,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,وذكرت وكالة الأنباء المحلية (جي.إن.إس) أن جماع...
3,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ووقع اختياره على واد عمقه 800 متر محاط بثماني ...
4,"أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",مسلح حوثي في إب وقال المصدر إن المسلحين الحوثي...


In [41]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [42]:
stop_words = set(stopwords.words('arabic'))
stemmer = ISRIStemmer()

In [43]:
def clean_text_arabic(text):
    # Remove unwanted characters and symbols
    text = re.sub(r"[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFBC2\uFBD3-\uFDFF\uFE70-\uFEFE]", " ", text)
    # Tokenization
    tokens = text.split()
    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]
    # Stemming
    tokens = [stemmer.stem(token) for token in tokens]
    # Join tokens back into a single string
    cleaned_text = ' '.join(tokens)
    return cleaned_text

In [44]:
def clean_text_data(df):
    df['summary'] = df['summary'].apply(clean_text_arabic)
    df['text'] = df['text'].apply(clean_text_arabic)
    return df

In [45]:
traindf = clean_text_data(train_df)
testdf = clean_text_data(test_df)
valdf = clean_text_data(val_df)

In [47]:
traindf

,summary,text
0,بدأ قوت اوكرانية سحب جزر قرم,وكان رئس وكر ؤقت، كسندر تورتشينوف، امر سحب قوت...
1,يتم غير عبر شهر ؤرخ يون هيرودو مصر هبة نيل ، س...,حلل عام حاج مصر لير متر كعب حصت حلي حسب خبراء ...
2,قلت شرط قطع هند قلم كشمير فجر قنبل ادى قتل عنص...,ذكر وكل باء حلة جي اس جمع جيش حمد تشدد اعل سؤل...
3,عام ، تجل رهب دعى كوكاي، حدر كثف شجر جبل كوا ك...,وقع خير واد عمق متر حاط بثم قمم حدة تشب ورق زه...
4,اكد صدر حرك همي أبناء حفظ حدد لبي سي قتل سلح ح...,سلح حوث اب وقل صدر سلح حوث هجم شطء حرك همي إفش...
...,...,...
37514,يدو بدن علق,يثر برك ولس تولباشيك شرق روس اضي وقد طلب سؤل س...
37515,سي رند قرر مي نعم,بين حدم ازم حجز رهن يد لجئ اير قهى بمد سيد است...
37516,دعي واصل,رمض اخر قبل سلم ظل كورو عام ثني ول، قبل سلم حا...
37517,حمل شمت,اصو ؤيد نظم عرب غلت قحم كونجرس نفر ديمقراطية و...
